# HW5 - Logistic Regression
##### 負責助教：張逸群 toosyou.tw@gmail.com

> 這次的作業將帶大家使用 Logistic Regression 進行乳癌是否為良性的預測

---
## 評分標準
* 上課繳交：25%
    * 在當天 16:20 前上傳至 new E3 作業，請將以下打包成 .zip 上傳：
        * 程式原始檔 (.ipynb）
        * 程式 .html 檔
* 程式分數：75%
    * 於一週內完成並上傳至 new E3
    
#### 注意：最終學期成績將視全班原始成績做調整

## 資料簡介
* Data - Breast cancer wisconsin (diagnostic) dataset
    * Number of Instances: 569
    * Number of Attributes: 30 numeric, predictive attributes and the class
    * Attribute Information:
        * radius (mean of distances from center to points on the perimeter)
        * texture (standard deviation of gray-scale values)
        * perimeter
        * area
        * smoothness (local variation in radius lengths)
        * compactness (perimeter^2 / area - 1.0)
        * concavity (severity of concave portions of the contour)
        * concave points (number of concave portions of the contour)
        * symmetry
        * fractal dimension (“coastline approximation” - 1)
        * The mean, standard error, and “worst” or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.
        * **class**:
            * WDBC-Malignant
            * WDBC-Benign
        
---

#### 姓名：陳盈均
#### 學號：109705011

## Packages that Might Be Used

In [94]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
%matplotlib inline

### 相關連結

* Packages
    * [numpy](http://www.numpy.org/)
    * [pandas](https://pandas.pydata.org/)
    * [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
    * [pyplot](https://matplotlib.org/tutorials/introductory/pyplot.html)
    * [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
    * [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
* Markdown
    * jupyter 可以支援 markdown 區塊，方便打上文字說明此份 notebook 在做的事情
    * [wiki](https://zh.wikipedia.org/wiki/Markdown)
    * [教學](https://markdown.tw/)

## Load Data - 20%

#### 請使用 `pd.read_csv` 將 `data.csv` 讀入成 `DataFrame` 至 `df` - 10%

In [95]:
df = pd.read_csv("data.csv")

In [96]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


#### 請將除了 `id` 和 `diagnosis` 外的 column 指派為 `X`（此為我們使用的 feature），並將 `diagnosis` column 指派為 `y`（此為我們將預測的目標）- 10%

In [97]:
X = df.iloc[:,2:-1]
y = df["diagnosis"]

In [98]:
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [99]:
y.head()

0    M
1    M
2    M
3    M
4    M
Name: diagnosis, dtype: object

## Train-Test Split - 10%

#### 請使用 `train_test_split` 將 `X` 與 `y` 分成 Training Set 以及 Testing Set - 10%

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)

## Logistic Regression - 30%

#### 請使用 [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 及 **`Training Set`** 對 [LogisticRegression Model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) 進行 [Hyperparamter Search](https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator) - 10%

> Model 的挑選嚴格禁止使用 testing set，否則無法保證挑出來的 model 對於新的資料是否還是適用

* 固定參數
    * tol: 0.01
    * penalty： l2
* Search 範圍
    * C: 1, 100
    * solver: saga lbfgs
    * warm_start: True, False
    

In [101]:
# turn off convergence warning 'cause it's annoying
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [102]:
parameters = {'tol':[0.01], 'penalty':["l2"], 'C':[1, 100], 'solver':["saga","lbfgs"], 'warm_start':[True, False]}
lr = LogisticRegression()
lr_grid = GridSearchCV(lr, param_grid = parameters).fit(X_train,y_train)

![meme1](https://imgur.com/QzAOB3V.jpg)

#### 將 Search 結果印出 - 0%

In [103]:
pd.DataFrame(lr_grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,param_tol,param_warm_start,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,0.055425,0.003078,0.004219,0.000746,1,l2,lbfgs,0.01,True,"{'C': 1, 'penalty': 'l2', 'solver': 'lbfgs', '...",0.935065,0.934211,0.960526,0.960526,0.894737,0.937013,0.024103,1
3,0.057315,0.007514,0.004181,0.000684,1,l2,lbfgs,0.01,False,"{'C': 1, 'penalty': 'l2', 'solver': 'lbfgs', '...",0.935065,0.934211,0.960526,0.960526,0.894737,0.937013,0.024103,1
6,0.059719,0.006592,0.004309,0.000510,100,l2,lbfgs,0.01,True,"{'C': 100, 'penalty': 'l2', 'solver': 'lbfgs',...",0.935065,0.934211,0.960526,0.960526,0.881579,0.934381,0.028830,3
7,0.051078,0.003015,0.003612,0.000373,100,l2,lbfgs,0.01,False,"{'C': 100, 'penalty': 'l2', 'solver': 'lbfgs',...",0.935065,0.934211,0.960526,0.960526,0.881579,0.934381,0.028830,3
0,0.023283,0.002226,0.003748,0.001056,1,l2,saga,0.01,True,"{'C': 1, 'penalty': 'l2', 'solver': 'saga', 't...",0.896104,0.894737,0.855263,0.907895,0.881579,0.887116,0.017977,5
1,0.022668,0.001824,0.003346,0.000391,1,l2,saga,0.01,False,"{'C': 1, 'penalty': 'l2', 'solver': 'saga', 't...",0.896104,0.894737,0.855263,0.907895,0.881579,0.887116,0.017977,5
4,0.027600,0.003249,0.004030,0.000641,100,l2,saga,0.01,True,"{'C': 100, 'penalty': 'l2', 'solver': 'saga', ...",0.896104,0.894737,0.855263,0.907895,0.881579,0.887116,0.017977,5
5,0.023958,0.003222,0.003400,0.000477,100,l2,saga,0.01,False,"{'C': 100, 'penalty': 'l2', 'solver': 'saga', ...",0.896104,0.894737,0.855263,0.907895,0.881579,0.887116,0.017977,5


#### 請回答以下問題 - 10%

1. 請問 Search 結果中，表現最好的是哪一組參數？ - 5%
    * 第六組
2. 承上，請問 `rank_test_score` 是以何作為依據？ - 5%
    * mean_test_score

#### 請使用表現最好的模型及 **testing set** 計算 **testing accuracy**

> 若在宣告 `lr_grid` 時其參數 `refit` 為 `True`，我們便可以使用 `lr_grid` 作為 search 範圍中最好的模型

In [104]:
print('lr acc:', lr_grid.score(X_test, y_test))

lr acc: 0.9680851063829787


#### 請回答以下問題 - 10%

1. 請問 `refit=True` 有什麼作用？
    * 按照最佳的參數配置重新訓練模型

## Prediction - 15%

#### 假設現在有一個新的病人資料如下，請使用 search 出最好的模型進行預測，並印出預測結果 - 10%

In [105]:
X_new = [11.61,
 16.02,
 75.46,
 408.2,
 0.1088,
 0.1168,
 0.07097,
 0.044969999999999996,
 0.1886,
 0.0632,
 0.2456,
 0.7339,
 1.6669999999999998,
 15.89,
 0.005884,
 0.02005,
 0.026310000000000004,
 0.013040000000000001,
 0.01848,
 0.001982,
 12.64,
 19.67,
 81.93,
 475.7,
 0.1415,
 0.217,
 0.2302,
 0.1105,
 0.2787,
 0.07427]

In [106]:
X_new = np.array(X_new).reshape(1,-1)
print('lr prediction:', lr_grid.predict(X_new))

lr prediction: ['B']


![ml_model](https://imgur.com/JsbckWU.jpg)

# 歡迎跟 ~~平常沒有人可以說話的~~ 助教縮縮話 <3

>　我今天考完微積分期中考了嗚呼!!!! 灑花!!!　你好你好

![](https://i.pinimg.com/originals/bc/30/39/bc303907006524200ac15cb2011e3050.gif)